# Lorenz System LSTM Predictor Demo

This notebook demonstrates the usage of the LSTM-based predictor for the Lorenz system.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from lorenz_lstm.lorenz_system import generate_lorenz_data
from lorenz_lstm.data import prepare_data_loaders
from lorenz_lstm.model import LorenzLSTM
from lorenz_lstm.train import train_model
from lorenz_lstm.visualization import plot_3d_trajectories, plot_time_series, plot_training_history

## 1. Generate Lorenz System Data

In [ ]:
# Parameters
t_span = (0, 30)
base_initial_xyz = [1, 1, 1]
num_trajectories = 200
perturbation_scale = 10
num_points = 3000

# Generate data
lorenz_data, t_eval = generate_lorenz_data(
    t_span, base_initial_xyz, num_trajectories,
    perturbation_scale, num_points
)

## 2. Prepare Data Loaders

In [ ]:
sequence_length = 10
batch_size = 500

train_loader, val_loader, test_loader = prepare_data_loaders(
    lorenz_data, sequence_length, batch_size
)

## 3. Train Model

In [ ]:
# Initialize model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = LorenzLSTM().to(device)

# Train
num_epochs = 70
history = train_model(model, train_loader, val_loader, num_epochs, device=device)

# Plot training history
plot_training_history(history)

## 4. Generate and Visualize Predictions

In [ ]:
# Parameters for prediction
num_test_trajectories = 3
num_predictions = 200

actual_trajectories = []
predicted_trajectories = []
initial_sequences = []

# Get test sequences
test_data = lorenz_data[-test_size:]
for i in range(num_test_trajectories):
    initial_sequence = torch.tensor(test_data[i][:sequence_length], dtype=torch.float32).unsqueeze(0)
    
    # We only need one sequence for prediction
    current_input = initial_sequence  # Keep batch dimension but use only first sequence
    
    # Store initial sequence without batch dimension for plotting
    initial_sequences.append(test_data[i][:sequence_length])
    
    # Generate predictions
    predictions = []
    
    for _ in range(num_predictions):
        with torch.no_grad():
            output = model(current_input)  # Shape: [1, 3]
            predictions.append(output[0].cpu().numpy())  # Get the prediction for the single sequence
            
            # Reshape output to match current_input dimensions [1, 1, 3]
            next_step = output.view(1, 1, 3)
            
            # Update input sequence for next prediction
            current_input = torch.cat([current_input[:, 1:], next_step], dim=1)
    
    predicted_trajectory = np.array(predictions)
    
    # Get corresponding actual trajectory
    actual_trajectory = test_data[i]  # Use last few trajectories
    actual_trajectory = actual_trajectory[:num_predictions + sequence_length]
    
    actual_trajectories.append(actual_trajectory)
    predicted_trajectories.append(predicted_trajectory)

# Convert to numpy arrays for consistent handling
actual_trajectories = np.array(actual_trajectories)
predicted_trajectories = np.array(predicted_trajectories)
initial_sequences = np.array(initial_sequences)

# Plot 3D trajectories
plot_3d_trajectories(actual_trajectories, predicted_trajectories, initial_sequences)

# Plot time series
plot_time_series(t_eval, actual_trajectories[0], predicted_trajectories[0], sequence_length)